<a href="https://colab.research.google.com/github/KimMinoo/Capstone-Design/blob/main/YOLOv3_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Train_YOLOv3 학습

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!git clone https://github.com/AlexeyAB/darknet

# change makefile to have GPU and OPENCV enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!make 

#!cp cfg/yolov3.cfg cfg/yolov3_training.cfg
!cp /content/gdrive/MyDrive/yolov3/data/yolov3_testing.cfg /content/darknet/cfg/yolov3_training.cfg

# Download weights darknet model 53
!wget https://pjreddie.com/media/files/darknet53.conv.74

!echo -e '' > data/obj.names
!echo -e '' > data/obj.data
!cp /content/drive/MyDrive/yolov3/data/obj.names /content/darknet/data/obj.names
!cp /content/drive/MyDrive/yolov3/data/obj.data /content/darknet/data/obj.data
!mkdir data/obj

import shutil
import os
src_dir = '/content/drive/MyDrive/yolov3/train_images'
dst_dir = '/content/darknet/data/obj'

# 디렉토리 내의 모든 파일을 이동합니다.
for filename in os.listdir(src_dir):
    # 파일 또는 디렉토리의 전체 경로를 구합니다.
    src = os.path.join(src_dir, filename)
    dst = os.path.join(dst_dir, filename)
    # 파일 또는 디렉토리를 이동합니다.
    shutil.move(src, dst)

import glob
images_list = glob.glob("/content/darknet/data/obj/*.jpg")
# images_list = glob.glob("data/obj/*.png")
print(images_list)

#Create training.txt file
train = int((len(images_list) / 10) * 8)
file = open("/content/darknet/data/train.txt", "w") 
file.write("\n".join(images_list[:train])) 
file.close() 

file = open("/content/darknet/data/test.txt", "w") 
file.write("\n".join(images_list[train:])) 
file.close() 

# Start the training 
!./darknet detector train data/obj.data cfg/yolov3_training.cfg darknet53.conv.74 -dont_show 

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
fatal: destination path 'darknet' already exists and is not an empty directory.
['/content/darknet/data/obj/1110021.jpg', '/content/darknet/data/obj/1110025.jpg', '/content/darknet/data/obj/1110035.jpg', '/content/darknet/data/obj/1110076.jpg', '/content/darknet/data/obj/1110038.jpg', '/content/darknet/data/obj/1110017.jpg', '/content/darknet/data/obj/1110066.jpg', '/content/darknet/data/obj/1110027.jpg', '/content/darknet/data/obj/1110051.jpg', '/content/darknet/data/obj/1110033.jpg', '/content/darknet/data/obj/1110055.jpg', '/content/darknet/data/obj/1110059.jpg', '/content/darknet/data/obj/1110014.jpg', '/content/darknet/data/obj/1110020.jpg', '/content/darknet/data/obj/1110074.jpg', '/content/darknet/data/obj/1110036.jpg', '/content/darknet/data/obj/1110047.jpg', '/content/darknet/data/obj/1110041.jpg', '/content/darknet/data/obj/1110070.jpg', '/content/d

Detection_YOLOv3 탐지

In [ ]:
import cv2
from google.colab.patches import cv2_imshow
#OpenCV는 이미지 및 비디오 처리, 객체 감지, 추적, 광학 문자 인식 등의 컴퓨터 비전
#애플리케이션을 개발할 때 널리 사용되는 오픈소스 라이브러리이다.

import numpy as np

net = cv2.dnn.readNet("/content/gdrive/MyDrive/yolov3/yolov3_training_final.weights", "/content/gdrive/MyDrive/yolov3/data/yolov3_testing.cfg")
# 다음 코드는 YOLOv3 객체 감지 알고리즘에 필요한 모델 가중치(weights) 파일과 모델 구성(cfg) 파일을
# 읽어와서 신경망 모델(net)을 생성하는 코드이다.

#'cv2,dnn' 모듈은 딥러닝 모델을 불러오고 실행하는 기능
#'readNet()' 함수는 모델 가중치 파일과 모델 구성 파일을 읽어와서 모델을 생성하는 함수

classes = []
with open("/content/gdrive/MyDrive/yolov3/data/obj.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]

layer_names = net.getLayerNames()
# 'getLayerNames()' 함수는 모델의 모든 레이어의 이름을 반환한다.

output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]
# 'getUnconnectedOutLayers()' 함수는 출력 레이어의 인덱스를 반환한다.
# Yolov3 모델의 경우, 출력 레이어는 세 개의 레이어로 구성된다.


colors = np.random.uniform(0, 255, size=(len(classes), 3))
# 객체 클래스에 대한 무작위 색상을 생성한다.
# 'np.random.uniform()' 함수는 지정된 범위 내에서 균등한 분포의 무작위 값을 생성한다.
# 따라서 위 코드는 객체 클래스 수(len(classes))에 맞게 0~255 범위의 무작위 색생을 생성하여 'color' 리스트에 저장한다.


img = cv2.imread("/content/drive/MyDrive/1110066/1110044.jpg")
# 위 코드는 파일을 읽어와서 이미지 객체(img)로 변환한다.
# cv2.imread()' 함수는 이미지 파일을 읽어와서 Numpy 배열 형태로 반환한다.

img = cv2.resize(img, None, fx=0.4, fy=0.4)
# 위 코드는 이미지 크기를 조정하는 함수이다.
# 'fx'와 'fy'는 각각 가로와 세로 방향의 스케일 비율이다.
# 위 같은 경우에는 0.4배로 축소되어 저장된다.

height, width, channels = img.shape

# 따라서 위 코드는 "image1.jpg" 파일을 읽어와서 크기를 조정하고, 'height', 'width', 'channels'
# 변수에 이미지의 높이, 너비, 채널 정보가 저장된다.

blob = cv2.dnn.blobFromImage(img, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
# 위 함수는 'img' 이미지를 모델에 입력 가능한 형태인 blob 형태로 변환한다.
# 입력 이미지를 전처리하여 CNN 모델의 입력으로 사용할 수 있는 형태로 반환한다.
# 이때 인자로 들어가는 '0.00392'는 픽셀값의 스케일링을 의미한다.
# Yolov3는 416x416 크기의 입력 이미지를 사용하므로 (416,416)으로 설정한다.
# (0,0,0)은 평균값(mean)을 의미한다.
# True 옵션은 이미지 채널을 BGR에서 RGB로 변경하는 것을 의미한다.
# crop=false는 이미지를 자르지 않는다는 것을 의미한다.

net.setInput(blob)
# 위 코드는 blob을 모델의 입력으로 설정한다.

outs = net.forward(output_layers)
# 입력된 이미지를 모델에 입력하고, 출력 레이어('output_layers')에 대한 결과를 반환한다.
# Yolov3 모델은 출력 레이어가 세 개로 구성되어 있기 때문에 리스트 형태로 결과값이 반환된다.

# 따라서 위 코드를 실행하면 'img' 이미지를 모델의 입력으로 사용하여 객체 감지를 실행하고,
# 'outs' 리스트에 출력 결과가 저장된다. 이후 이 출력 결과를 가시화하여 객체 감지 결과를 확인할 수 있다.

class_ids = []
confidences = []
boxes = []
for out in outs:
    for detection in out:
        scores = detection[5:]
        class_id = np.argmax(scores)
        confidence = scores[class_id]
        if confidence > 0.5:
            center_x = int(detection[0] * width)
            center_y = int(detection[1] * height)
            w = int(detection[2] * width)
            h = int(detection[3] * height)            
            x = int(center_x - w / 2)
            y = int(center_y - h / 2)
            boxes.append([x, y, w, h])
            confidences.append(float(confidence))
            class_ids.append(class_id)


indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
# 위 코드는 Non-Maximum Suppression 알고리즘을 사용하여 박스를 필터링하고 겹치는 박스들 중
# 가장 확률이 높은 박스만 남기는 역할을 한다.
# boxes: 경계 상자들의 좌표값들이 담긴 리스트
# confidences: 검출된 경계상자들의 신뢰도 값 리스트
# score_threshold: 박스의 신뢰도 값이 이 값보다 작은 박스는 제거한다.
# nms_threshold: 겹치는 영역의 IOU 값이 이 값보다 큰 박스들을 제거한다.

font = cv2.FONT_HERSHEY_PLAIN
for i in range(len(boxes)):
    if i in indexes:
        x, y, w, h = boxes[i]
        label = str(classes[class_ids[i]])
        color = colors[class_ids[i]]
        cv2.rectangle(img, (x, y), (x + w, y + h), color, 2)
        cv2.putText(img, label, (x, y + 30), font, 3, color, 3)
cv2_imshow(img)
cv2.waitKey(0)
cv2.destroyAllWindows()


#-------------------------------------------------------------------------------------------
#여러개의 이미지를 한 번에 Detecting하기
# import cv2
# import numpy as np

# net = cv2.dnn.readNet("/content/gdrive/MyDrive/yolov3/yolov3_last_training.weights", "/content/gdrive/MyDrive/yolov3/data/yolov3_testing.cfg")
# classes = []
# with open("/content/drive/MyDrive/yolov3/data/obj.names", "r") as f:
#     classes = [line.strip() for line in f.readlines()]
# layer_names = net.getLayerNames()
# output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]
# colors = np.random.uniform(0, 255, size=(len(classes), 3))

# import glob
# img_list = glob.glob("/content/darknet/data/obj/train_image/*.jpg")
# images=[]
# height=[]
# width=[]
# channels=[]
# for i in range(len(img_list)):
#   img = cv2.imread(str(img_list[i]))
#   print(img)
#   img = cv2.resize(img, None, fx=0.4, fy=0.4)
#   h, w, ch = img.shape
#   images.append(img)
#   height.append(h)
#   width.append(w)
#   channels.append(ch)

# result=[]

# for i in range(len(images)):
#   blob = cv2.dnn.blobFromImage(images[i], 0.00392, (416, 416), (0, 0, 0), True, crop=False)
#   net.setInput(blob)
#   outs = net.forward(output_layers)
#   result.append(outs)

# final_class_ids = []
# final_confidences = []
# final_boxes = []

# for i,outs in enumerate(result):
#   class_ids = []
#   confidences = []
#   boxes = []
#   for out in outs:
#       for detection in out:
#           scores = detection[5:]
#           class_id = np.argmax(scores)
#           confidence = scores[class_id]
#           if confidence > 0.5:
#               center_x = int(detection[0] * width[i])
#               center_y = int(detection[1] * height[i])
#               w = int(detection[2] * width[i])
#               h = int(detection[3] * height[i])            
#               x = int(center_x - w / 2)
#               y = int(center_y - h / 2)
#               boxes.append([x, y, w, h])
#               confidences.append(float(confidence))
#               class_ids.append(class_id)
  
#   final_class_ids.append(class_ids)
#   final_confidences.append(confidences)
#   final_boxes.append(boxes)

# final_indexes=[]

# for i in range(len(final_boxes)):
#   indexes = cv2.dnn.NMSBoxes(final_boxes[i], final_confidences[i], 0.5, 0.4)
#   final_indexes.append(indexes)


# from google.colab.patches import cv2_imshow

# for j in range(len(final_indexes)):
#   font = cv2.FONT_HERSHEY_PLAIN
#   indexes = final_indexes[j]
#   boxes = final_boxes[j]
#   class_ids = final_class_ids[j]
#   img = images[j]

#   for i in range(len(boxes)):
#       if i in indexes:
#         x, y, w, h = boxes[i]
#         label = str(classes[class_ids[i]])
#         color = colors[class_ids[i]]
#         cv2.rectangle(img, (x, y), (x + w, y + h), color, 2)
#         cv2.putText(img, label, (x, y + 30), font, 3, color, 3)
#   cv2_imshow(img)
#   cv2.waitKey(0)
#   cv2.destroyAllWindows()


Tesseract를 이용하여 text 추출

In [ ]:
!sudo apt install tesseract-ocr
!pip install pytesseract

!wget https://github.com/tesseract-ocr/tessdata/raw/main/kor.traineddata -P /usr/share/tesseract-ocr/4.00/tessdata
import os
os.environ['TESSDATA_PREFIX'] = '/usr/share/tesseract-ocr/4.00/tessdata'

import cv2
import pytesseract

# 이미지 열기
img = cv2.imread('/content/gdrive/MyDrive/1110066/1110044.jpg')

img = cv2.resize(img, None, interpolation=cv2.INTER_LINEAR)

# 이미지 출력
cv2_imshow(img)
cv2.waitKey(0)

# 좌표 지정
x1s=[]
x2s=[]
y1s=[]
y2s=[]

for i in range(len(boxes)):
    if i in indexes:
      x1, y1, w, h = boxes[i]
      x2 = x1 + w
      y2 = y1 + h
      x1s.append(x1)
      y1s.append(y1)
      x2s.append(x2)
      y2s.append(y2)

# 추출할 영역 자르기
crop_imgs=[]
for j in range(len(x1s)):
  crop_img = img[y1s[j]:y2s[j], x1s[j]:x2s[j]]
  crop_imgs.append(crop_img)

for m in range(len(crop_imgs)):
  cv2_imshow(crop_imgs[m])
  cv2.waitKey(0)
  cv2.destroyAllWindows()


# 추출할 영역을 그레이스케일로 변환
gray_imgs=[]
for k in range(len(crop_imgs)):
  gray_img = cv2.cvtColor(crop_imgs[k], cv2.COLOR_BGR2GRAY)
  gray_imgs.append(gray_img)

# 추출할 영역에서 문자 추출
texts=[]
for t in range(len(crop_imgs)):
  text = pytesseract.image_to_string(gray_imgs[t],lang='kor+eng')
  texts.append(text)

for i in range(len(texts)):
  print('추출된 텍스트:', texts[i])

